## PARCEIROS 360

###  Pagamento - Utilização Factoring & Bloqueios

- <b>Analista(s)</b>: Daniel Rezende
- <b>Projeto</b>: Parceiros 360 - Um modelo de análise de risco para auxiliar no controle das tomadas de decisões através de análises preditivas
- <b>Data</b>: 10/12/2022
- <b>Descrição</b>: 

In [5]:
!python --version

Python 3.7.12


### BIBLIOTECAS:

In [6]:
!pip install treeinterpreter

In [7]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from treeinterpreter import treeinterpreter as ti

## alertas
import warnings
warnings.filterwarnings("ignore")

### LAYOUT:

In [8]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 16,
    'axes.titlesize': 18, 'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(rc=rc)
sns.set_style("darkgrid")
sns.set_palette("pastel")

### CONFIGS:

In [9]:
#pd.set_option('max_columns', 200)
pd.options.display.max_columns = 200 

seed = 42
np.random.seed(seed)

### FUNÇÕES

In [10]:
#SE PRECISAR DO ONE HOT USAR ANTES DO SCALERS PRA NAO DAR ERRO

def scalers(base):
    nova_base = base.copy()
    scaler = MinMaxScaler(feature_range=(0,1))
    for column in base.columns:
        aux = scaler.fit_transform(base[[column]])
        nova_base[column] = aux
    return nova_base

In [11]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET

def makedf(base_treino_scaled, dados, kpi_nome, base_treino):
    
    kmeans = KMeans(n_clusters = 3, max_iter = 300, init = 'k-means++', n_init = 10, random_state = 0)
    clusters = kmeans.fit_predict(base_treino_scaled)

    model = RandomForestClassifier(random_state=0)
    model.fit(base_treino_scaled,clusters)
    prediction, bias, contributions = ti.predict(model, base_treino_scaled)

    contributions = np.round(contributions,5)*1000

    contributions_c1 = []
    contributions_c2 = []
    contributions_c3 = []
    contributions_filtered = []

    for i in range(len(contributions)):
        contributions_c1.append([contributions[i][0][0], contributions[i][1][0], contributions[i][2][0], contributions[i][3][0], contributions[i][4][0], contributions[i][5][0]])
        contributions_c2.append([contributions[i][0][1], contributions[i][1][1], contributions[i][2][1], contributions[i][3][1], contributions[i][4][1], contributions[i][5][1]])
        contributions_c3.append([contributions[i][0][2], contributions[i][1][2], contributions[i][2][2], contributions[i][3][2], contributions[i][4][2], contributions[i][5][2]])
    
    for i in range(len(clusters)):
        if clusters[i] == 0:
            contributions_filtered.append(contributions_c1[i])
        if clusters[i] == 1:
            contributions_filtered.append(contributions_c2[i])
        if clusters[i] == 2:
            contributions_filtered.append(contributions_c3[i])

    scaler = MinMaxScaler(feature_range=(0,100))
    contributions_filtered_scaled = scaler.fit_transform(contributions_filtered)

    contributions_filtered = pd.DataFrame(contributions_filtered_scaled, columns=base_treino_scaled.columns + '_CONTRIBUTION')

    base_treino['KPI_original'] = dados[kpi_nome].values
    base_treino['resultado_modelo'] = clusters
    base_treino['resultado_modelo'] = base_treino['resultado_modelo'].map({0:'BOM', 1:'MÉDIO', 2:'RUIM'})

    base_treino.reset_index(inplace=True, drop=False)

    results = pd.concat([base_treino, contributions_filtered], axis=1)
    return results

### DADOS:

In [12]:
PATH = "datasets/"

****
### BASE DE PAGAMENTO:

In [13]:
FILE = "/kaggle/input/datasetsnew/previsao_pagamentos.xlsx"
# FILE = "previsao_pagamentos.csv"

pagto = pd.read_excel(FILE)
pagto['RAIZ_CNPJ'] = pagto['RAIZ_CNPJ'].apply(str)
pagto = pagto.rename(columns={'resultado_modelo': 'resultado_modelo_pagto'})
pagto = pagto[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_pagto']].drop_duplicates().reset_index(drop=True)

pagto

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_pagto
0,2641663,2021-01-01,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,RUIM
1,2641663,2021-02-01,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,RUIM
2,2641663,2021-03-01,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,RUIM
3,2641663,2021-04-01,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,RUIM
4,2641663,2021-05-01,FUNDAÃÃO CPQD - CENTRO DE PESQUISA,RUIM
...,...,...,...,...
4511,29090742,2022-03-01,M2 SERVICOS DE MENSAGERIA,RUIM
4512,29090742,2022-04-01,M2 SERVICOS DE MENSAGERIA,RUIM
4513,29090742,2022-05-01,M2 SERVICOS DE MENSAGERIA,RUIM
4514,29090742,2022-06-01,M2 SERVICOS DE MENSAGERIA,RUIM


***
### BASE DE FATURAMENTO:

In [14]:
FILE = "/kaggle/input/datasetsnew/previsao_faturamento.xlsx"
# FILE = "previsao_pagamentos.csv"

faturamento = pd.read_excel(FILE)
faturamento['RAIZ_CNPJ'] = faturamento['RAIZ_CNPJ'].apply(str)
faturamento = faturamento.rename(columns={'resultado_modelo': 'resultado_modelo_faturamento'})
faturamento = faturamento[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_faturamento']]\
                           .drop_duplicates().reset_index(drop=True)

faturamento

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_faturamento
0,0,2021-01-01,HODNET S/A,MEDIO
1,0,2021-02-01,HODNET S/A,MEDIO
2,0,2021-03-01,HODNET S/A,MEDIO
3,0,2021-04-01,HODNET S/A,MEDIO
4,0,2021-05-01,HODNET S/A,MEDIO
...,...,...,...,...
8333,96534094,2022-03-01,ACCENTURE DO BRASIL LTDA,MEDIO
8334,96534094,2022-04-01,ACCENTURE DO BRASIL LTDA,MEDIO
8335,96534094,2022-05-01,ACCENTURE DO BRASIL LTDA,MEDIO
8336,96534094,2022-06-01,ACCENTURE DO BRASIL LTDA,MEDIO


***
### BASE COMISSÃO

In [15]:
FILE = "/kaggle/input/datasetsnew/previsao_comissao.xlsx"
# FILE = "previsao_comissao.csv"

comissao = pd.read_excel(FILE)
comissao['RAIZ_CNPJ'] = comissao['RAIZ_CNPJ'].apply(str)
comissao = comissao.rename(columns={'resultado_modelo': 'resultado_modelo_comissao'})
comissao = comissao[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_comissao']]\
                     .drop_duplicates().reset_index(drop=True)

comissao

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_comissao
0,78599,2021-01-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,RISCO MÉDIO
1,78599,2021-02-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,RISCO MÉDIO
2,78599,2021-03-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,RISCO MÉDIO
3,78599,2021-04-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,RISCO MÉDIO
4,78599,2021-05-01,GERENCIAL BRASIL PARTICIPAÃÃES E EM,RISCO MÉDIO
...,...,...,...,...
10143,46298215,2022-07-01,SOUZA E SOUZA INTER. EM TELEFONIA L,RISCO MÉDIO
10144,46337103,2022-06-01,R E COM E SER DE TELEFONIA LTDA,RISCO MÉDIO
10145,46337103,2022-07-01,R E COM E SER DE TELEFONIA LTDA,RISCO MÉDIO
10146,46354697,2022-06-01,ALLIANCE TELECOMUNICACOES BRASIL LT,RISCO MÉDIO


***
### BASE ALIADOS

In [16]:
FILE = "/kaggle/input/datasetsnew/previsao_aliados_variacao_kpi1.xlsx"
# FILE = "previsao_aliados_variacao_kpi1.csv"

previsao_aliados_variacao_kpi1 = pd.read_excel(FILE)
previsao_aliados_variacao_kpi1['COMPETENCIA'] = pd.to_datetime(previsao_aliados_variacao_kpi1['COMPETENCIA'])
previsao_aliados_variacao_kpi1 = previsao_aliados_variacao_kpi1.rename(columns={'resultado_modelo': 'resultado_modelo_previsao_aliados_variacao_kpi1'})
previsao_aliados_variacao_kpi1 = previsao_aliados_variacao_kpi1[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_previsao_aliados_variacao_kpi1']]\
                                                               .drop_duplicates().reset_index(drop=True)

previsao_aliados_variacao_kpi1

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_variacao_kpi1
0,00000000,2021-01-01,HODNET S/A,BOM
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,BOM
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",BOM
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,BOM
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,BOM
...,...,...,...,...
25786,96662168,2022-07-01,DREBES E CIA LTDA,BOM
25787,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,BOM
25788,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",BOM
25789,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",BOM


In [17]:
FILE = "/kaggle/input/datasetsnew/previsao_aliados_trabalhista_kpi2.xlsx"
# FILE = "previsao_aliados_trabalhista_kpi2.csv"

previsao_aliados_trabalhista_kpi2 = pd.read_excel(FILE)
previsao_aliados_trabalhista_kpi2['COMPETENCIA'] = pd.to_datetime(previsao_aliados_trabalhista_kpi2['COMPETENCIA'])
previsao_aliados_trabalhista_kpi2 = previsao_aliados_trabalhista_kpi2.rename(columns={'resultado_modelo': 'resultado_modelo_previsao_aliados_trabalhista_kpi2'})
previsao_aliados_trabalhista_kpi2 = previsao_aliados_trabalhista_kpi2[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_previsao_aliados_trabalhista_kpi2']]\
                                                                     .drop_duplicates().reset_index(drop=True)

previsao_aliados_trabalhista_kpi2

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_trabalhista_kpi2
0,00000000,2021-01-01,HODNET S/A,BOM
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,BOM
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",BOM
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,BOM
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,BOM
...,...,...,...,...
25786,96662168,2022-07-01,DREBES E CIA LTDA,BOM
25787,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,BOM
25788,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",BOM
25789,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",BOM


In [18]:
FILE = "/kaggle/input/datasetsnew/previsao_aliados_endividamento_kpi3.xlsx"
# FILE = "previsao_aliados_endividamento_kpi3.csv"

previsao_aliados_endividamento_kpi3 = pd.read_excel(FILE)
previsao_aliados_endividamento_kpi3['COMPETENCIA'] = pd.to_datetime(previsao_aliados_endividamento_kpi3['COMPETENCIA'])
previsao_aliados_endividamento_kpi3 = previsao_aliados_endividamento_kpi3.rename(columns={'resultado_modelo': 'resultado_modelo_previsao_aliados_endividamento_kpi3'})
previsao_aliados_endividamento_kpi3 = previsao_aliados_endividamento_kpi3[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_previsao_aliados_endividamento_kpi3']]\
                                                                         .drop_duplicates().reset_index(drop=True)

previsao_aliados_endividamento_kpi3

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_endividamento_kpi3
0,00000000,2021-01-01,HODNET S/A,BOM
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,BOM
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",BOM
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,BOM
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,BOM
...,...,...,...,...
25786,96662168,2022-07-01,DREBES E CIA LTDA,BOM
25787,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,BOM
25788,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",BOM
25789,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",BOM


In [19]:
FILE = "/kaggle/input/datasetsnew/20221122 - Avaliacao_Consolidada_indicadores.xlsx"
# FILE = "previsao_aliados_endividamento_kpi3.csv"

Avaliacao_Consolidada_indicadores = pd.read_excel(FILE)
Avaliacao_Consolidada_indicadores = Avaliacao_Consolidada_indicadores[['RAIZ_CNPJ', 'Nota_Final_Indic']]

Avaliacao_Consolidada_indicadores['RAIZ_CNPJ'] = Avaliacao_Consolidada_indicadores['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
Avaliacao_Consolidada_indicadores = Avaliacao_Consolidada_indicadores[Avaliacao_Consolidada_indicadores["RAIZ_CNPJ"].str.contains("NAN")== False]
Avaliacao_Consolidada_indicadores = Avaliacao_Consolidada_indicadores[Avaliacao_Consolidada_indicadores["RAIZ_CNPJ"].str.contains("-")== False]\
                                                                         .drop_duplicates().reset_index(drop=True)

Avaliacao_Consolidada_indicadores.columns

Index(['RAIZ_CNPJ', 'Nota_Final_Indic'], dtype='object')

***
### JOINS:

In [20]:
for_pgm = previsao_aliados_variacao_kpi1.merge(previsao_aliados_trabalhista_kpi2, 
                                               on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                                               how='left')\
                                        .drop_duplicates().reset_index(drop=True)

for_pgm

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_variacao_kpi1,resultado_modelo_previsao_aliados_trabalhista_kpi2
0,00000000,2021-01-01,HODNET S/A,BOM,BOM
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,BOM,BOM
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",BOM,BOM
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,BOM,BOM
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,BOM,BOM
...,...,...,...,...,...
25786,96662168,2022-07-01,DREBES E CIA LTDA,BOM,BOM
25787,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,BOM,BOM
25788,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",BOM,BOM
25789,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",BOM,BOM


In [21]:
for_pgm_com = for_pgm.merge(previsao_aliados_endividamento_kpi3,                       
                            on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                            how='left')\
                     .drop_duplicates().reset_index(drop=True)
for_pgm_com

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_variacao_kpi1,resultado_modelo_previsao_aliados_trabalhista_kpi2,resultado_modelo_previsao_aliados_endividamento_kpi3
0,00000000,2021-01-01,HODNET S/A,BOM,BOM,BOM
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,BOM,BOM,BOM
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",BOM,BOM,BOM
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,BOM,BOM,BOM
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,BOM,BOM,BOM
...,...,...,...,...,...,...
25786,96662168,2022-07-01,DREBES E CIA LTDA,BOM,BOM,BOM
25787,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,BOM,BOM,BOM
25788,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",BOM,BOM,BOM
25789,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",BOM,BOM,BOM


In [22]:
for_pgm_com_ali1 = for_pgm_com.merge(comissao, 
                                     on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                                     how='left')\
                              .drop_duplicates().reset_index(drop=True)

for_pgm_com_ali1

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_variacao_kpi1,resultado_modelo_previsao_aliados_trabalhista_kpi2,resultado_modelo_previsao_aliados_endividamento_kpi3,resultado_modelo_comissao
0,00000000,2021-01-01,HODNET S/A,BOM,BOM,BOM,NaN
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,BOM,BOM,BOM,NaN
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",BOM,BOM,BOM,NaN
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,BOM,BOM,BOM,NaN
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,BOM,BOM,BOM,NaN
...,...,...,...,...,...,...,...
25805,96662168,2022-07-01,DREBES E CIA LTDA,BOM,BOM,BOM,RISCO ALTO
25806,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,BOM,BOM,BOM,RISCO ALTO
25807,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",BOM,BOM,BOM,NaN
25808,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",BOM,BOM,BOM,NaN


In [23]:
for_pgm_com_ali2 = for_pgm_com_ali1.merge(faturamento, 
                                          on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                                          how='left')\
                                   .drop_duplicates().reset_index(drop=True)

for_pgm_com_ali2

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_variacao_kpi1,resultado_modelo_previsao_aliados_trabalhista_kpi2,resultado_modelo_previsao_aliados_endividamento_kpi3,resultado_modelo_comissao,resultado_modelo_faturamento
0,00000000,2021-01-01,HODNET S/A,BOM,BOM,BOM,NaN,NaN
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,BOM,BOM,BOM,NaN,NaN
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",BOM,BOM,BOM,NaN,NaN
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,BOM,BOM,BOM,NaN,NaN
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,BOM,BOM,BOM,NaN,NaN
...,...,...,...,...,...,...,...,...
25805,96662168,2022-07-01,DREBES E CIA LTDA,BOM,BOM,BOM,RISCO ALTO,NaN
25806,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,BOM,BOM,BOM,RISCO ALTO,NaN
25807,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",BOM,BOM,BOM,NaN,NaN
25808,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",BOM,BOM,BOM,NaN,NaN


In [24]:
for_pgm_com_ali3a = for_pgm_com_ali2.merge(pagto, 
                                          on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                                          how='left')\
                                   .drop_duplicates().reset_index(drop=True)

for_pgm_com_ali3a

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_variacao_kpi1,resultado_modelo_previsao_aliados_trabalhista_kpi2,resultado_modelo_previsao_aliados_endividamento_kpi3,resultado_modelo_comissao,resultado_modelo_faturamento,resultado_modelo_pagto
0,00000000,2021-01-01,HODNET S/A,BOM,BOM,BOM,NaN,NaN,NaN
1,00000B60,2021-01-01,ITP TECNOLOGICA SL,BOM,BOM,BOM,NaN,NaN,NaN
2,00000B86,2021-01-01,"DELOITTE ADVISORY, S.L.",BOM,BOM,BOM,NaN,NaN,NaN
3,00028986,2021-01-01,ELEVADORES ATLAS SCHINDLER SA,BOM,BOM,BOM,NaN,NaN,NaN
4,00059799,2021-01-01,HDS SISTEMAS DE ENERGIA LTDA,BOM,BOM,BOM,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
25824,96662168,2022-07-01,DREBES E CIA LTDA,BOM,BOM,BOM,RISCO ALTO,NaN,NaN
25825,98102924,2022-07-01,DELTASUL UTILIDADES LTDA,BOM,BOM,BOM,RISCO ALTO,NaN,NaN
25826,B8677373,2022-07-01,"NETMETRIX SOLUTIONS, S.L.",BOM,BOM,BOM,NaN,NaN,NaN
25827,B9898262,2022-07-01,"KENMEI TECHNOLOGIES, S.L.",BOM,BOM,BOM,NaN,NaN,NaN


In [25]:
for_pgm_com_ali3 = for_pgm_com_ali3a.merge(Avaliacao_Consolidada_indicadores, 
                                          on=['RAIZ_CNPJ'], 
                                          how='inner')\
                                   .drop_duplicates().reset_index(drop=True)

for_pgm_com_ali3

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_variacao_kpi1,resultado_modelo_previsao_aliados_trabalhista_kpi2,resultado_modelo_previsao_aliados_endividamento_kpi3,resultado_modelo_comissao,resultado_modelo_faturamento,resultado_modelo_pagto,Nota_Final_Indic
0,00000000,2021-01-01,HODNET S/A,BOM,BOM,BOM,NaN,NaN,NaN,RUIM
1,00000000,2021-02-01,HODNET S/A,MEDIO,MEDIO,BOM,NaN,NaN,NaN,RUIM
2,00000000,2021-03-01,HODNET S/A,MEDIO,MEDIO,BOM,NaN,NaN,NaN,RUIM
3,00000000,2021-04-01,HODNET S/A,BOM,BOM,BOM,NaN,NaN,NaN,RUIM
4,00000000,2021-05-01,HODNET S/A,BOM,BOM,BOM,NaN,NaN,NaN,RUIM
...,...,...,...,...,...,...,...,...,...,...
23903,37415516,2022-07-01,TAS - TECNOLOGIA APLICADA A SEGURAN,BOM,BOM,BOM,NaN,NaN,NaN,MÉDIO
23904,44378838,2022-07-01,NK 147 EMPREENDIMENTOS E PARTICIPAÃ,BOM,BOM,BOM,NaN,NaN,NaN,SEM AVALIAÇÃO
23905,59844621,2022-07-01,MICROFONE COMUNICAÃÃES ARTISTICAS L,BOM,BOM,BOM,NaN,NaN,NaN,SEM AVALIAÇÃO
23906,73092132,2022-07-01,RBC INFORMAÃÃES ELETRÃNICAS,BOM,BOM,BOM,NaN,NaN,NaN,SEM AVALIAÇÃO


#### JOIN pagamento vs faturamento:

#### JOIN pagamento vs faturamento vs comissao:

#### JOIN pagamento vs faturamento vs comissao vs aliados

In [26]:
# fill string 0 para NaN
for_pgm_com_ali3['resultado_modelo_pagto'] = for_pgm_com_ali3['resultado_modelo_pagto'].fillna(value='NAO AVALIADO')
for_pgm_com_ali3['resultado_modelo_faturamento'] = for_pgm_com_ali3['resultado_modelo_faturamento'].fillna(value='NAO AVALIADO')
for_pgm_com_ali3['resultado_modelo_comissao'] = for_pgm_com_ali3['resultado_modelo_comissao'].fillna(value='NAO AVALIADO')
for_pgm_com_ali3['resultado_modelo_previsao_aliados_variacao_kpi1'] = for_pgm_com_ali3['resultado_modelo_previsao_aliados_variacao_kpi1'].fillna(value='NAO AVALIADO')
for_pgm_com_ali3['resultado_modelo_previsao_aliados_trabalhista_kpi2'] = for_pgm_com_ali3['resultado_modelo_previsao_aliados_trabalhista_kpi2'].fillna(value='NAO AVALIADO')
for_pgm_com_ali3['resultado_modelo_previsao_aliados_endividamento_kpi3'] = for_pgm_com_ali3['resultado_modelo_previsao_aliados_endividamento_kpi3'].fillna(value='NAO AVALIADO')

for_pgm_com_ali3

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_previsao_aliados_variacao_kpi1,resultado_modelo_previsao_aliados_trabalhista_kpi2,resultado_modelo_previsao_aliados_endividamento_kpi3,resultado_modelo_comissao,resultado_modelo_faturamento,resultado_modelo_pagto,Nota_Final_Indic
0,00000000,2021-01-01,HODNET S/A,BOM,BOM,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,RUIM
1,00000000,2021-02-01,HODNET S/A,MEDIO,MEDIO,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,RUIM
2,00000000,2021-03-01,HODNET S/A,MEDIO,MEDIO,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,RUIM
3,00000000,2021-04-01,HODNET S/A,BOM,BOM,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,RUIM
4,00000000,2021-05-01,HODNET S/A,BOM,BOM,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,RUIM
...,...,...,...,...,...,...,...,...,...,...
23903,37415516,2022-07-01,TAS - TECNOLOGIA APLICADA A SEGURAN,BOM,BOM,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,MÉDIO
23904,44378838,2022-07-01,NK 147 EMPREENDIMENTOS E PARTICIPAÃ,BOM,BOM,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,SEM AVALIAÇÃO
23905,59844621,2022-07-01,MICROFONE COMUNICAÃÃES ARTISTICAS L,BOM,BOM,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,SEM AVALIAÇÃO
23906,73092132,2022-07-01,RBC INFORMAÃÃES ELETRÃNICAS,BOM,BOM,BOM,NAO AVALIADO,NAO AVALIADO,NAO AVALIADO,SEM AVALIAÇÃO


***
### ORDINAL ENCODING

In [27]:
from sklearn.preprocessing import OrdinalEncoder

for_pgm_com_ali3_oe = OrdinalEncoder().fit_transform(for_pgm_com_ali3[['resultado_modelo_pagto', 
                                                                       'resultado_modelo_faturamento', 
                                                                       'resultado_modelo_comissao', 
                                                                       'resultado_modelo_previsao_aliados_variacao_kpi1', 
                                                                       'resultado_modelo_previsao_aliados_trabalhista_kpi2', 
                                                                       'resultado_modelo_previsao_aliados_endividamento_kpi3']])

for_pgm_com_ali3['resultado_modelo_pagto_oe'] = for_pgm_com_ali3_oe[:, 0]

for_pgm_com_ali3['resultado_modelo_faturamento_oe'] = for_pgm_com_ali3_oe[:, 1]

for_pgm_com_ali3['resultado_modelo_comissao_oe'] = for_pgm_com_ali3_oe[:, 2]

for_pgm_com_ali3['resultado_modelo_previsao_aliados_variacao_kpi1_oe'] = for_pgm_com_ali3_oe[:, 3]

for_pgm_com_ali3['resultado_modelo_previsao_aliados_trabalhista_kpi2_oe'] = for_pgm_com_ali3_oe[:, 4]

for_pgm_com_ali3['resultado_modelo_previsao_aliados_endividamento_kpi3_oe'] = for_pgm_com_ali3_oe[:, 5]

In [28]:
dados = for_pgm_com_ali3.copy()
dados.set_index(['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], inplace=True)
dados.head(5)

resultado_modelo_previsao_aliados_variacao_kpi1 resultado_modelo_previsao_aliados_trabalhista_kpi2 resultado_modelo_previsao_aliados_endividamento_kpi3 resultado_modelo_comissao resultado_modelo_faturamento resultado_modelo_pagto Nota_Final_Indic  resultado_modelo_pagto_oe  resultado_modelo_faturamento_oe  resultado_modelo_comissao_oe  resultado_modelo_previsao_aliados_variacao_kpi1_oe  resultado_modelo_previsao_aliados_trabalhista_kpi2_oe  resultado_modelo_previsao_aliados_endividamento_kpi3_oe
RAIZ_CNPJ COMPETENCIA NOME_FORNECEDOR                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
00000000  2021-01-01  HODNET S/A                                                  BOM                                                BOM                                                BOM                NAO AVALIADO                 NAO AVALIADO           NAO AVALIADO             RUIM                        2.0                              2.0                           0.0                                                0.0                                                 0.0                                                    0.0      
          2021-02-01  HODNET S/A                                                MEDIO                                              MEDIO                                                BOM                NAO AVALIADO                 NAO AVALIADO           NAO AVALIADO             RUIM                        2.0                              2.0                           0.0                                                1.0                                                 1.0                                                    0.0      
          2021-03-01  HODNET S/A                                                MEDIO                                              MEDIO                                                BOM                NAO AVALIADO                 NAO AVALIADO           NAO AVALIADO             RUIM                        2.0                              2.0                           0.0                                                1.0                                                 1.0                                                    0.0      
          2021-04-01  HODNET S/A                                                  BOM                                                BOM                                                BOM                NAO AVALIADO                 NAO AVALIADO           NAO AVALIADO             RUIM                        2.0                              2.0                           0.0                                                0.0                                                 0.0                                                    0.0      
          2021-05-01  HODNET S/A                                                  BOM                                                BOM                                                BOM                NAO AVALIADO                 NAO AVALIADO           NAO AVALIADO             RUIM                        2.0                              2.0                           0.0                                                0.0                                                 0.0                                                    0.0

In [29]:
dados.columns

Index(['resultado_modelo_previsao_aliados_variacao_kpi1', 'resultado_modelo_previsao_aliados_trabalhista_kpi2', 'resultado_modelo_previsao_aliados_endividamento_kpi3', 'resultado_modelo_comissao', 'resultado_modelo_faturamento', 'resultado_modelo_pagto', 'Nota_Final_Indic', 'resultado_modelo_pagto_oe', 'resultado_modelo_faturamento_oe', 'resultado_modelo_comissao_oe', 'resultado_modelo_previsao_aliados_variacao_kpi1_oe', 'resultado_modelo_previsao_aliados_trabalhista_kpi2_oe', 'resultado_modelo_previsao_aliados_endividamento_kpi3_oe'], dtype='object')

In [30]:
base_treino_KPI_A1_variacao = dados[['resultado_modelo_pagto_oe', 
                                     'resultado_modelo_faturamento_oe', 
                                     'resultado_modelo_comissao_oe', 
                                     'resultado_modelo_previsao_aliados_variacao_kpi1_oe', 
                                     'resultado_modelo_previsao_aliados_trabalhista_kpi2_oe', 
                                     'resultado_modelo_previsao_aliados_endividamento_kpi3_oe']]

In [31]:
base_treino_KPI_A1_variacao_scaled = scalers(base_treino_KPI_A1_variacao)

base_treino_KPI_A1_variacao_scaled

resultado_modelo_pagto_oe  resultado_modelo_faturamento_oe  resultado_modelo_comissao_oe  resultado_modelo_previsao_aliados_variacao_kpi1_oe  resultado_modelo_previsao_aliados_trabalhista_kpi2_oe  resultado_modelo_previsao_aliados_endividamento_kpi3_oe
RAIZ_CNPJ COMPETENCIA NOME_FORNECEDOR                                                                                                                                                                                                                                                                                    
00000000  2021-01-01  HODNET S/A                                              0.666667                         0.666667                           0.0                                                0.0                                                 0.0                                                    0.0      
          2021-02-01  HODNET S/A                                              0.666667                         0.666667                           0.0                                                0.5                                                 0.5                                                    0.0      
          2021-03-01  HODNET S/A                                              0.666667                         0.666667                           0.0                                                0.5                                                 0.5                                                    0.0      
          2021-04-01  HODNET S/A                                              0.666667                         0.666667                           0.0                                                0.0                                                 0.0                                                    0.0      
          2021-05-01  HODNET S/A                                              0.666667                         0.666667                           0.0                                                0.0                                                 0.0                                                    0.0      
...                                                                                ...                              ...                           ...                                                ...                                                 ...                                                    ...      
37415516  2022-07-01  TAS - TECNOLOGIA APLICADA A SEGURAN                     0.666667                         0.666667                           0.0                                                0.0                                                 0.0                                                    0.0      
44378838  2022-07-01  NK 147 EMPREENDIMENTOS E PARTICIPAÃ                    0.666667                         0.666667                           0.0                                                0.0                                                 0.0                                                    0.0      
59844621  2022-07-01  MICROFONE COMUNICAÃÃES ARTISTICAS L                   0.666667                         0.666667                           0.0                                                0.0                                                 0.0                                                    0.0      
73092132  2022-07-01  RBC INFORMAÃÃES ELETRÃNICAS                          0.666667                         0.666667                           0.0                                                0.0                                                 0.0                                                    0.0      
73163529  2022-07-01  DPZ&T COMUNICAÃÃES LTDA                               1.000000                         1.000000                           0.0                                                0.0                                                 0.0                                         

***
### CLUSTERING COM K-Means:

In [32]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI, scaled
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET
#base_treino_KPI_A1_variacao = BASE DE TREINO, SEM A COLUNA DO KPI


results_kpi = makedf(base_treino_KPI_A1_variacao_scaled, dados, 'Nota_Final_Indic', base_treino_KPI_A1_variacao)
results_kpi

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_pagto_oe,resultado_modelo_faturamento_oe,resultado_modelo_comissao_oe,resultado_modelo_previsao_aliados_variacao_kpi1_oe,resultado_modelo_previsao_aliados_trabalhista_kpi2_oe,resultado_modelo_previsao_aliados_endividamento_kpi3_oe,KPI_original,resultado_modelo,resultado_modelo_pagto_oe_CONTRIBUTION,resultado_modelo_faturamento_oe_CONTRIBUTION,resultado_modelo_comissao_oe_CONTRIBUTION,resultado_modelo_previsao_aliados_variacao_kpi1_oe_CONTRIBUTION,resultado_modelo_previsao_aliados_trabalhista_kpi2_oe_CONTRIBUTION,resultado_modelo_previsao_aliados_endividamento_kpi3_oe_CONTRIBUTION
0,00000000,2021-01-01,HODNET S/A,2.0,2.0,0.0,0.0,0.0,0.0,RUIM,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
1,00000000,2021-02-01,HODNET S/A,2.0,2.0,0.0,1.0,1.0,0.0,RUIM,BOM,52.219690,58.975036,1.524692,98.308791,94.397394,7.572689
2,00000000,2021-03-01,HODNET S/A,2.0,2.0,0.0,1.0,1.0,0.0,RUIM,BOM,52.219690,58.975036,1.524692,98.308791,94.397394,7.572689
3,00000000,2021-04-01,HODNET S/A,2.0,2.0,0.0,0.0,0.0,0.0,RUIM,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
4,00000000,2021-05-01,HODNET S/A,2.0,2.0,0.0,0.0,0.0,0.0,RUIM,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23903,37415516,2022-07-01,TAS - TECNOLOGIA APLICADA A SEGURAN,2.0,2.0,0.0,0.0,0.0,0.0,MÉDIO,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
23904,44378838,2022-07-01,NK 147 EMPREENDIMENTOS E PARTICIPAÃ,2.0,2.0,0.0,0.0,0.0,0.0,SEM AVALIAÇÃO,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
23905,59844621,2022-07-01,MICROFONE COMUNICAÃÃES ARTISTICAS L,2.0,2.0,0.0,0.0,0.0,0.0,SEM AVALIAÇÃO,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
23906,73092132,2022-07-01,RBC INFORMAÃÃES ELETRÃNICAS,2.0,2.0,0.0,0.0,0.0,0.0,SEM AVALIAÇÃO,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838


In [33]:
results_kpi = results_kpi.drop_duplicates().reset_index(drop=True)
results_kpi

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,resultado_modelo_pagto_oe,resultado_modelo_faturamento_oe,resultado_modelo_comissao_oe,resultado_modelo_previsao_aliados_variacao_kpi1_oe,resultado_modelo_previsao_aliados_trabalhista_kpi2_oe,resultado_modelo_previsao_aliados_endividamento_kpi3_oe,KPI_original,resultado_modelo,resultado_modelo_pagto_oe_CONTRIBUTION,resultado_modelo_faturamento_oe_CONTRIBUTION,resultado_modelo_comissao_oe_CONTRIBUTION,resultado_modelo_previsao_aliados_variacao_kpi1_oe_CONTRIBUTION,resultado_modelo_previsao_aliados_trabalhista_kpi2_oe_CONTRIBUTION,resultado_modelo_previsao_aliados_endividamento_kpi3_oe_CONTRIBUTION
0,00000000,2021-01-01,HODNET S/A,2.0,2.0,0.0,0.0,0.0,0.0,RUIM,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
1,00000000,2021-02-01,HODNET S/A,2.0,2.0,0.0,1.0,1.0,0.0,RUIM,BOM,52.219690,58.975036,1.524692,98.308791,94.397394,7.572689
2,00000000,2021-03-01,HODNET S/A,2.0,2.0,0.0,1.0,1.0,0.0,RUIM,BOM,52.219690,58.975036,1.524692,98.308791,94.397394,7.572689
3,00000000,2021-04-01,HODNET S/A,2.0,2.0,0.0,0.0,0.0,0.0,RUIM,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
4,00000000,2021-05-01,HODNET S/A,2.0,2.0,0.0,0.0,0.0,0.0,RUIM,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23903,37415516,2022-07-01,TAS - TECNOLOGIA APLICADA A SEGURAN,2.0,2.0,0.0,0.0,0.0,0.0,MÉDIO,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
23904,44378838,2022-07-01,NK 147 EMPREENDIMENTOS E PARTICIPAÃ,2.0,2.0,0.0,0.0,0.0,0.0,SEM AVALIAÇÃO,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
23905,59844621,2022-07-01,MICROFONE COMUNICAÃÃES ARTISTICAS L,2.0,2.0,0.0,0.0,0.0,0.0,SEM AVALIAÇÃO,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838
23906,73092132,2022-07-01,RBC INFORMAÃÃES ELETRÃNICAS,2.0,2.0,0.0,0.0,0.0,0.0,SEM AVALIAÇÃO,MÉDIO,48.903905,60.256241,17.375385,36.871564,33.758958,7.931838


In [34]:
results_kpi.to_excel(r'previsao_all.xlsx')